In [1]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:

indir = "../../../resulting data/"
outdir = "../../../bible_map/202011 GeoBrowser"


In [3]:
verses_features = pd.read_parquet(indir + "verses_features.parquet")

In [7]:
verses_features

,bible_identifier,bible_book,bible_chapter,bible_book_chapter,relations,monogamy,poligamy,marriage,second-marriage,second-marriage-divorce,...,#per129,pla15,#pla74,pla69,org435,org297,org183,org102,#pla75,org22
b.GEN.001.001,b.GEN.001.001,GEN,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b.GEN.001.002,b.GEN.001.002,GEN,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b.GEN.001.003,b.GEN.001.003,GEN,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b.GEN.001.004,b.GEN.001.004,GEN,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b.GEN.001.005,b.GEN.001.005,GEN,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b.REV.022.017,b.REV.022.017,REV,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b.REV.022.018,b.REV.022.018,REV,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b.REV.022.019,b.REV.022.019,REV,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b.REV.022.020,b.REV.022.020,REV,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
verses_features.columns.tolist()[0:100]

['bible_identifier',
 'bible_book',
 'bible_chapter',
 'bible_book_chapter',
 'relations',
 'monogamy',
 'poligamy',
 'marriage',
 'second-marriage',
 'second-marriage-divorce',
 'second-marriage-widow',
 'interfaith-marriage',
 'interfaith-relation',
 'non-marital-relation',
 'marital-relation-political-family',
 'lord-servant-relation',
 'marital-relation-direct-family',
 'arranged-marriage',
 'divorce',
 'engaged',
 'love',
 'jealousy',
 'ascendant-descendant',
 'indirect-ascendant-descendant',
 'acts',
 'social-related-act',
 'wedding',
 'body-related-act',
 'massage',
 'kiss',
 'hug',
 'dance',
 'abortion',
 'circumcision',
 'breast-feeding',
 'sexual-act',
 'sexual-act-stage',
 'seduction',
 'erotic-time',
 'initation-sexual-act',
 'orgasm',
 'ejaculation',
 'time-after-sex',
 'sexual-act-type',
 'coitus',
 'first-sexual-act',
 'coitus-interruptus',
 'self-masturbation',
 'masturbation',
 'oral-sex',
 'anal-sex',
 'non-penetrative-sex',
 'female-rape',
 'male-rape',
 'sexual-assa

In [5]:
verses_features["bible_book"] = verses_features["bible_identifier"].str.extract("b\.(...).+$")

In [8]:
verses_features["bible_book_chapter"] = verses_features["bible_identifier"].str.extract("b\.(...\.\d+).+$")

In [9]:
verses_features["bible_chapter"] = verses_features["bible_identifier"].str.extract("b\....\.(\d+).+$")

In [10]:
verses_features["bible_book"]

b.GEN.001.001    GEN
b.GEN.001.002    GEN
b.GEN.001.003    GEN
b.GEN.001.004    GEN
b.GEN.001.005    GEN
                ... 
b.REV.022.017    REV
b.REV.022.018    REV
b.REV.022.019    REV
b.REV.022.020    REV
b.REV.022.021    REV
Name: bible_book, Length: 31065, dtype: object

In [6]:
columns_places = [column for column in verses_features.columns.tolist() if "#pla" in column]

In [11]:
verses_features_places = verses_features[['bible_identifier',
 'bible_book',
 'bible_chapter',
 'bible_book_chapter',]+columns_places ]

In [12]:
results = []
for index_verse, values_verse in verses_features_places[columns_places].iterrows():
    #print(index_verse, values_verse.sum())
    if values_verse.sum() == 0:
        results.append([index_verse, ""])
    else:
        #print(values_verse.dropna())
        for index_column, column in values_verse.dropna().iteritems():
            results.append([index_verse, index_column])

results_df = pd.DataFrame(results, columns = ["verse", "geolocation_id"])       


In [13]:
results_df

,verse,geolocation_id
0,b.GEN.001.001,#pla41
1,b.GEN.001.002,#pla41
2,b.GEN.001.003,
3,b.GEN.001.004,
4,b.GEN.001.005,
...,...,...
35858,b.REV.022.017,
35859,b.REV.022.018,
35860,b.REV.022.019,#pla71
35861,b.REV.022.020,


In [14]:
results_df.to_csv(outdir + "verses_id-place.tsv", sep="\t")

In [15]:
entities = pd.read_excel("../../../entities.xls")

In [16]:
entities

,order,id,NormalizedName-sp,aclaration,variants,comments,book,cert,wikidata id,cert_wikidata,...,PHM,HEB,JAM,1PE,2PE,1JO,2JO,3JO,JUD,REV
0,2,#per0,persona no definida,NaN,NaN,NaN,NT,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,#per1,Jesús,NaN,NaN,NaN,NT,NaN,Q302,NaN,...,13.0,75.0,5.0,34.0,24.0,48.0,7.0,0.0,8.0,102.0
2,4,#per2,Abraham,NaN,Abram,NaN,AT,NaN,Q9181,NaN,...,0.0,13.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,#per3,Isaac,NaN,NaN,NaN,AT,NaN,Q671872,NaN,...,0.0,7.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,#per4,Jacob,NaN,Israel,NaN,AT,NaN,Q289957,NaN,...,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983,3985,#wor17,libro de Isaías,NaN,NaN,NaN,NaN,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3984,3986,#wor18,libros de Lamentos,NaN,NaN,NaN,NaN,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3985,3987,#ani0,animales,NaN,NaN,NaN,NaN,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3986,3988,#thin0,cosa,NaN,NaN,NaN,NaN,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
entities.loc[entities["geo_cert"].isin(["medium", "high"])]

,order,id,NormalizedName-sp,aclaration,variants,comments,book,cert,wikidata id,cert_wikidata,...,PHM,HEB,JAM,1PE,2PE,1JO,2JO,3JO,JUD,REV
2901,2903,#pla1,Belén,NaN,NaN,NaN,NaN,NaN,https://www.wikidata.org/wiki/Special:EntityPa...,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2902,2904,#pla2,Judea,NaN,Judá,Es lo mismo que Judá?,NaN,NaN,Q104028,NaN,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2903,2905,#pla3,Jerusalén,NaN,NaN,Ciudad de David? Llamada Aholiba en EZE 23,NaN,NaN,Q1218,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2904,2906,#pla4,Israel,NaN,NaN,NaN,NaN,NaN,Q3185305,NaN,...,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2905,2907,#pla5,Egipto,NaN,Cam,NaN,NaN,NaN,Q79,NaN,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,3643,#pla741,Susa,NaN,NaN,NaN,NaN,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3676,3678,#pla776,tierra de los amorreos,NaN,NaN,NaN,NaN,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3685,3687,#pla785,Lais,NaN,la rebautizan como Dan,NaN,NaN,medium,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3753,3755,#pla853,Askenaz,NaN,NaN,NaN,NaN,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
verses_places_longitude_latitude = pd.merge(results_df, entities.loc[entities["geo_cert"].isin(["medium", "high"])][["id", "NormalizedName-sp", "longitude", "latitude"]], left_on="geolocation_id", right_on="id", how="left")

In [66]:
verses_places_longitude_latitude

,verse,geolocation_id,id,NormalizedName-sp,longitude,latitude
0,b.GEN.001.001,#pla41,#pla41,tierra,0.0,-90.0
1,b.GEN.001.002,#pla41,#pla41,tierra,0.0,-90.0
2,b.GEN.001.003,,NaN,NaN,NaN,NaN
3,b.GEN.001.004,,NaN,NaN,NaN,NaN
4,b.GEN.001.005,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
35858,b.REV.022.017,,NaN,NaN,NaN,NaN
35859,b.REV.022.018,,NaN,NaN,NaN,NaN
35860,b.REV.022.019,#pla71,NaN,NaN,NaN,NaN
35861,b.REV.022.020,,NaN,NaN,NaN,NaN


In [67]:
verses_places_longitude_latitude["id"] = verses_places_longitude_latitude.index

In [68]:
verses_places_longitude_latitude["book"] = verses_places_longitude_latitude["verse"].str.extract(r"b\.(...).+$")

In [69]:
verses_places_longitude_latitude.head(50)

,verse,geolocation_id,id,NormalizedName-sp,longitude,latitude,book
0,b.GEN.001.001,#pla41,0,tierra,0.0,-90.0,GEN
1,b.GEN.001.002,#pla41,1,tierra,0.0,-90.0,GEN
2,b.GEN.001.003,,2,NaN,NaN,NaN,GEN
3,b.GEN.001.004,,3,NaN,NaN,NaN,GEN
4,b.GEN.001.005,,4,NaN,NaN,NaN,GEN
5,b.GEN.001.006,,5,NaN,NaN,NaN,GEN
6,b.GEN.001.007,,6,NaN,NaN,NaN,GEN
7,b.GEN.001.008,,7,NaN,NaN,NaN,GEN
8,b.GEN.001.009,,8,NaN,NaN,NaN,GEN
9,b.GEN.001.010,,9,NaN,NaN,NaN,GEN


In [70]:
verses_places_longitude_latitude.to_csv(outdir + "/verses_places_longitude_latitude.tsv", sep="\t")

In [71]:
verses_places_longitude_latitude.to_csv(outdir + "/verses_places_longitude_latitude.csv")

In [72]:
verses_places_longitude_latitude

,verse,geolocation_id,id,NormalizedName-sp,longitude,latitude,book
0,b.GEN.001.001,#pla41,0,tierra,0.0,-90.0,GEN
1,b.GEN.001.002,#pla41,1,tierra,0.0,-90.0,GEN
2,b.GEN.001.003,,2,NaN,NaN,NaN,GEN
3,b.GEN.001.004,,3,NaN,NaN,NaN,GEN
4,b.GEN.001.005,,4,NaN,NaN,NaN,GEN
...,...,...,...,...,...,...,...
35858,b.REV.022.017,,35858,NaN,NaN,NaN,REV
35859,b.REV.022.018,,35859,NaN,NaN,NaN,REV
35860,b.REV.022.019,#pla71,35860,NaN,NaN,NaN,REV
35861,b.REV.022.020,,35861,NaN,NaN,NaN,REV


In [73]:
verses_places_longitude_latitude["Name"] = verses_places_longitude_latitude["NormalizedName-sp"]

In [74]:
verses_places_longitude_latitude["Address"] = verses_places_longitude_latitude["NormalizedName-sp"]

In [75]:
verses_places_longitude_latitude["Description"] = verses_places_longitude_latitude["verse"]

In [76]:
verses_places_longitude_latitude["Longitude"] = verses_places_longitude_latitude["longitude"]

In [77]:
verses_places_longitude_latitude["Latitude"] = verses_places_longitude_latitude["latitude"]

In [78]:
verses_places_longitude_latitude["TimeStamp"] = verses_places_longitude_latitude["id"]

In [79]:
verses_places_longitude_latitude["TimeSpan:begin"] = ""

In [80]:
verses_places_longitude_latitude["TimeSpan:end"] = ""

In [81]:
verses_places_longitude_latitude["GettyID"] = ""

In [82]:
selected_columns = ["Name","Address","Description","Longitude","Latitude","TimeStamp","TimeSpan:begin","TimeSpan:end","GettyID"]




In [84]:
verses_places_longitude_latitude[selected_columns].to_csv(outdir + "/verses_places_selected_columns.csv")

In [85]:
verses_places_longitude_latitude.index = verses_places_longitude_latitude["verse"]

In [86]:
verses_places_longitude_latitude.head()

,verse,geolocation_id,id,NormalizedName-sp,longitude,latitude,book,Name,Address,Description,Longitude,Latitude,TimeStamp,TimeSpan:begin,TimeSpan:end,GettyID
verse,,,,,,,,,,,,,,,,
b.GEN.001.001,b.GEN.001.001,#pla41,0,tierra,0.0,-90.0,GEN,tierra,tierra,b.GEN.001.001,0.0,-90.0,0,,,
b.GEN.001.002,b.GEN.001.002,#pla41,1,tierra,0.0,-90.0,GEN,tierra,tierra,b.GEN.001.002,0.0,-90.0,1,,,
b.GEN.001.003,b.GEN.001.003,,2,NaN,NaN,NaN,GEN,NaN,NaN,b.GEN.001.003,NaN,NaN,2,,,
b.GEN.001.004,b.GEN.001.004,,3,NaN,NaN,NaN,GEN,NaN,NaN,b.GEN.001.004,NaN,NaN,3,,,
b.GEN.001.005,b.GEN.001.005,,4,NaN,NaN,NaN,GEN,NaN,NaN,b.GEN.001.005,NaN,NaN,4,,,


In [107]:
verses_places_longitude_latitude.dropna()[selected_columns+["id"]].to_csv(outdir + "/verses_places_selected_columns_without_na.csv")

In [88]:
books = pd.read_excel("../../../documentation/books.xlsx")

In [95]:
books

,Unnamed: 0,bible,id,title_Spanish,testament,encoded,title_English,codebook,chaptersc,genre,latitude,longitude,topical bible,wikid,sexual_annotation
0,0,bible,1,Génesis,old,1,Genesis,GEN,50,historical,32.615687,36.793083,Gen,Q9184,1.0
1,1,bible,2,Éxodo,old,1,Exodus,EXO,40,historical,30.469969,33.843280,Exod,Q9190,NaN
2,2,bible,3,Levítico,old,1,Leviticus,LEV,27,law,30.521765,34.112561,Lev,Q41490,NaN
3,3,bible,4,Números,old,1,Numbers,NUM,36,law,31.221618,34.841433,Num,Q43099,NaN
4,4,bible,5,Deuteronomio,old,1,Deuteronomy,DEU,34,law,31.506106,35.735247,Deut,Q42614,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,bible,62,1 Juan,new,1,1 John,1JO,5,letter,NaN,NaN,1John,Q131101,NaN
62,62,bible,63,2 Juan,new,1,2 John,2JO,1,letter,NaN,NaN,2John,Q131453,NaN
63,63,bible,64,3 Juan,new,1,3 John,3JO,1,letter,NaN,NaN,3John,Q131462,NaN
64,64,bible,65,Judas,new,1,Jude,JUD,1,letter,26.820553,30.802498,Jude,Q131466,NaN


In [102]:
verses_places_longitude_latitude = pd.merge(verses_places_longitude_latitude, books[["genre", "codebook"]], left_on="book", right_on="codebook", how="left")


In [103]:
verses_places_longitude_latitude

,verse,geolocation_id,id,NormalizedName-sp,longitude,latitude,book,Name,Address,Description,Longitude,Latitude,TimeStamp,TimeSpan:begin,TimeSpan:end,GettyID,genre,codebook
0,b.GEN.001.001,#pla41,0,tierra,0.0,-90.0,GEN,tierra,tierra,b.GEN.001.001,0.0,-90.0,0,,,,historical,GEN
1,b.GEN.001.002,#pla41,1,tierra,0.0,-90.0,GEN,tierra,tierra,b.GEN.001.002,0.0,-90.0,1,,,,historical,GEN
2,b.GEN.001.003,,2,NaN,NaN,NaN,GEN,NaN,NaN,b.GEN.001.003,NaN,NaN,2,,,,historical,GEN
3,b.GEN.001.004,,3,NaN,NaN,NaN,GEN,NaN,NaN,b.GEN.001.004,NaN,NaN,3,,,,historical,GEN
4,b.GEN.001.005,,4,NaN,NaN,NaN,GEN,NaN,NaN,b.GEN.001.005,NaN,NaN,4,,,,historical,GEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35858,b.REV.022.017,,35858,NaN,NaN,NaN,REV,NaN,NaN,b.REV.022.017,NaN,NaN,35858,,,,apocalyptic,REV
35859,b.REV.022.018,,35859,NaN,NaN,NaN,REV,NaN,NaN,b.REV.022.018,NaN,NaN,35859,,,,apocalyptic,REV
35860,b.REV.022.019,#pla71,35860,NaN,NaN,NaN,REV,NaN,NaN,b.REV.022.019,NaN,NaN,35860,,,,apocalyptic,REV
35861,b.REV.022.020,,35861,NaN,NaN,NaN,REV,NaN,NaN,b.REV.022.020,NaN,NaN,35861,,,,apocalyptic,REV


In [108]:
for genre in verses_places_longitude_latitude.genre.unique():
    verses_places_longitude_latitude.loc[verses_places_longitude_latitude["genre"] == genre].dropna()[selected_columns+["id"]].to_csv(outdir + "/verses_places_selected_columns_without_na_" + genre + ".csv")